In [ ]:

import pandas as pd
import numpy as np
import string, textblob, re
import pickle
import nltk 

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from unidecode import unidecode
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras import layers, models, optimizers
from keras.preprocessing import text, sequence
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

df = pd.read_csv('./data/out_Tax.csv')

In [ ]:
df.head()

df = df.dropna()
df = df[df["Text"].str.contains("Duitse vertaling")==False]
df

In [ ]:
data_text = df[['Text']]
type(data_text)

In [ ]:
def remove_accented_chars(text):
    text = unidecode(text)
    return text

def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result

def remove_slash_with_space(text): 
    return text.replace('\\', " ")

def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

def text_lowercase(text): 
    return text.lower()  

def remove_whitespace(text): 
    return  " ".join(text.split()) 

def remove_stopwords(text):
    #nltk.download('stopwords')
    stop_words = stopwords.words('dutch')
    nstopword = ['bis', 'NL', 'FR', 'artikel','wijziging','http','kbwib', 'koninklijk','einde' ,'koninklijke', 'minister' ,'koning', 'besluit', 'belgisch',
                 'tabel' ,'definitief', 'ministrieel', 'staatsblad', 'oktober', 'september','november','december','januari','februari',
                 'maart', 'april','mei','juli', 'juni', 'augustus', 'begin','navigatie', 'eerste', 'woord', 'laatste', 'woord',
                 'publicatie', 'numac','wwwejusticejustfgovbeelibesluitstaatsblad', 'belgielexbe', 'ministerie', 'ministerieel','identificatiecode','kruispuntbank', 'wetgeving', 'raad']
    stop_words.extend(nstopword)
    #print('++++++',stop_words)
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if (len(word) >3) and word not in stop_words] 
    return ' '.join(filtered_text)

def stem_words(text): 
    stemmer = PorterStemmer() 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    return ' '.join(stems)

def lemmatize_words(text): 
    lemmatizer = WordNetLemmatizer() 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    return ' '.join(lemmas)


In [ ]:
# Perform preprocessing

def perform_preprocessing(text):
    
    text = remove_accented_chars(text)
    text = remove_numbers(text)
    text = text_lowercase(text)
    text = remove_stopwords(text)
    text = remove_slash_with_space(text)
    text = remove_punctuation(text)
    # text = stem_words(text)
    text = remove_whitespace(text)
    text = lemmatize_words(text)
    text = remove_stopwords(text)
    return text

text_sample= df['Text']
type(text_sample)

df['Lema'] = df['Text'].apply(perform_preprocessing)
print(df['Lema'][28])
print('++++++++++++++++++++++++++++++++++++++++++++++')
text = lemmatize_words(df['Lema'][28])
text = remove_stopwords(text)
df.head() 

In [ ]:
df['Lema'][25]

In [ ]:

df.head()
df = df[df.Num_words != 0]
#df = df[df.Num_words != 37]

#l = df[df['Num_words'] < 100]

#findex = df['Lema'][24].index('numac')
#sindex = len(df['Lema'][25]) - df['Lema'][25].reverse().index('begin')-1
df.describe()


In [ ]:
df['Lema'][28]

In [ ]:
df['Text'][28]

In [ ]:
import yake

doc = df['Lema'][37]

kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(doc)
for kw in keywords:
  print(kw)


In [ ]:
from rake_nltk import Rake
r = Rake()
my_text = """
When it comes to evaluating the performance of keyword extractors, you can use some of the standard metrics in machine learning: accuracy, precision, recall, and F1 score. However, these metrics don’t reflect partial matches; they only consider the perfect match between an extracted segment and the correct prediction for that tag.
Fortunately, there are some other metrics capable of capturing partial matches. An example of this is ROUGE.
"""
ss = df['Lema'][48][250:3700]
print(len(ss))
r.extract_keywords_from_text(ss)
keywordList           = []
rankedList            = r.get_ranked_phrases_with_scores()
for keyword in rankedList:
  keyword_updated       = keyword[1].split()
  keyword_updated_string    = " ".join(keyword_updated[:2])
  keywordList.append(keyword_updated_string)
  if(len(keywordList)>9):
    break
print(keywordList)

In [ ]:
import spacy
l = spacy.explain('PROPN')
from spacy.lang.nl.examples import sentences 

nlp = spacy.load("nl_core_news_sm")

def get_hotwords(text):
    result = []
    pos_tag = ['ADJ', 'NOUN'] 
    doc = nlp(text)
    for token in doc:
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return ' '.join(result)
df['Sum'] = df['Lema'].apply(get_hotwords)
df["Num_words"] = df['Lema'].apply(lambda x: len(x))

df.head()

In [ ]:
df.describe()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)


In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
doc_sample = df['Text'][28]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [ ]:
l = df[['Lema']]
dictionary = gensim.corpora.Dictionary(l)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break